In [1]:
using CLOUD
using LinearAlgebra
sequence_path = "../results/advection_collapsed_2D_refine_p3_upwind/"

θ = π/4
a = sqrt(2)
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ))))  # end time
initial_data = InitialDataSine(A,k)
conservation_law = linear_advection_equation((a*cos(θ),a*sin(θ)),λ=1.0);

In [2]:
p = 3
p_map = 1
form = WeakConservationForm(StandardMapping())

reference_approximation =ReferenceApproximation(
    DGSEM(p), CollapsedTri(),
    volume_quadrature_rule=(LGQuadrature(),LGRQuadrature()),
    facet_quadrature_rule=(LGQuadrature(),LGRQuadrature()), 
    chain_rule_diff=true, mapping_degree=p_map,
    N_plot=15)

sequence = [2, 4, 8, 16, 32, 64]

mesh_gen_func(M) = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M), collapsed=true), 
    reference_approximation.reference_element, 0.15)

dt_func(M) = 0.05*(L/M)/(a*(2*p+1));

In [3]:
run_refinement(conservation_law, reference_approximation, 
    initial_data, form, Lazy(), (0.0,T), sequence, 
    mesh_gen_func, dt_func, sequence_path, n_s=10)

LoadError: InterruptException: